<a href="https://colab.research.google.com/github/kahlilXhub/Data-Cleaning-Pandas/blob/master/Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import numpy as np

In [119]:
smartphone = pd.read_csv("smartphones - smartphones.csv")

In [120]:
smartphone = smartphone[smartphone['ram'].str.contains('RAM',case=False, na=False)]

In [121]:
smartphone.head(5)

,model,price,rating,sim,processor,ram,battery,display,camera,card,os
0,OnePlus 11 5G,"₹54,999",89.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi, NFC","Snapdragon 8 Gen2, Octa Core, 3.2 GHz Processor","12 GB RAM, 256 GB inbuilt",5000 mAh Battery with 100W Fast Charging,"6.7 inches, 1440 x 3216 px, 120 Hz Display wit...",50 MP + 48 MP + 32 MP Triple Rear & 16 MP Fron...,Memory Card Not Supported,Android v13
1,OnePlus Nord CE 2 Lite 5G,"₹19,989",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 33W Fast Charging,"6.59 inches, 1080 x 2412 px, 120 Hz Display wi...",64 MP + 2 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
2,Samsung Galaxy A14 5G,"₹16,499",75.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Exynos 1330, Octa Core, 2.4 GHz Processor","4 GB RAM, 64 GB inbuilt",5000 mAh Battery with 15W Fast Charging,"6.6 inches, 1080 x 2408 px, 90 Hz Display with...",50 MP + 2 MP + 2 MP Triple Rear & 13 MP Front ...,"Memory Card Supported, upto 1 TB",Android v13
3,Motorola Moto G62 5G,"₹14,999",81.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Snapdragon 695, Octa Core, 2.2 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with Fast Charging,"6.55 inches, 1080 x 2400 px, 120 Hz Display wi...",50 MP + 8 MP + 2 MP Triple Rear & 16 MP Front ...,"Memory Card (Hybrid), upto 1 TB",Android v12
4,Realme 10 Pro Plus,"₹24,999",82.0,"Dual Sim, 3G, 4G, 5G, VoLTE, Wi-Fi","Dimensity 1080, Octa Core, 2.6 GHz Processor","6 GB RAM, 128 GB inbuilt",5000 mAh Battery with 67W Fast Charging,"6.7 inches, 1080 x 2412 px, 120 Hz Display wit...",108 MP + 8 MP + 2 MP Triple Rear & 16 MP Front...,Memory Card Not Supported,Android v13


In [122]:
smartphone['brand'] = smartphone['model'].str.strip().str.extract(r'(\w+)')

In [123]:
smartphone['price_cleaned'] = smartphone['price'].str.replace(r'[^\d.]', '',regex=True)
smartphone['price_cleaned'] = pd.to_numeric(smartphone['price_cleaned'] , errors='coerce')
smartphone['price_usd'] = smartphone['price_cleaned'] / 90

In [124]:
smartphone.drop(columns=['price_cleaned'], inplace=True)

In [125]:
smartphone[['ram_temp','storage']] = smartphone['ram'].str.split(',',expand=True)
smartphone['ram_gb'] = smartphone['ram_temp'].str.extract(r'(\d+)').astype(float)
smartphone['storage_type'] = smartphone['storage']
smartphone['storage_gb'] = smartphone['storage'].str.extract(r'(\d)').astype(float)

In [126]:
storage = smartphone['storage'].str.extract(r'(\d+\.?\d*)\s*(MB|GB|TB)')
storage.columns = ['value','unit']
storage['value'] = storage['value'].astype(float)

size_storage = [
    storage['unit'] == 'TB',
    storage['unit'] == 'MB'
]

conversion_storage = [
    storage['value'] * 1024,
    storage['value'] / 1024
]

smartphone['storage_gb'] = np.select(size_storage,conversion_storage,default=storage['value'])
storage_median = smartphone['storage_gb'].median()
smartphone['storage_gb'] = smartphone['storage_gb'].fillna(storage_median)

In [127]:
smartphone.drop(columns=['ram_temp','storage','storage_type'], inplace=True)

In [128]:
slot_card = smartphone['card'].str.extract(r'(\d+\.?\d*)\s*(MB|GB|TB)')
slot_card.columns = ['value','unit']
slot_card['value'] = pd.to_numeric(slot_card['value'], errors='coerce')

size_card = [
    slot_card['unit'] == 'TB',
   slot_card['unit'] == 'MB'
]

conversion_card = [
    slot_card['value'] * 1024,
    slot_card['value'] / 1024
]

smartphone['card_gb'] = np.select(size_card,conversion_card,default=0)
card_median = smartphone['card_gb'].median()
smartphone['card_gb'] = smartphone['card_gb'].fillna(card_median)

In [129]:
smartphone['has_5g'] = smartphone['sim'].str.contains(r'5G', case=False).astype(int)
smartphone['has_nfc'] = smartphone['sim'].str.contains(r'NFC', case=False).astype(int)
type_sim = {
    'Dual':2,
    'Single':1,
    'Triple':3
}
smartphone['total_sim'] = smartphone['sim'].str.extract(r'(Dual|Single|Triple)')[0].map(type_sim)
sim_median = smartphone['total_sim'].median()
smartphone['total_sim'] = smartphone['total_sim'].fillna(sim_median).astype(int)

In [130]:
type_core = {
    'Deca':10,
    'Octa':8,
    'Hexa': 6,
    'Quad': 4,
    'Dual': 2,
    'Single': 1
}
brand_pattern = r'(Apple|Bionic|Snapdragon|Dimensity|Helio|Exynos|Kirin|Tiger|Unisoc|Spreadtrum|Google|Tensor)'
smartphone['processor_name'] = smartphone['processor'].str.extract(f'({brand_pattern})', flags=re.IGNORECASE)[0].str.strip().fillna('Unknown')
smartphone['cores'] = smartphone['processor'].str.extract(r'(\w+)\s*Core')[0].str.strip()
smartphone['cores'] = smartphone['cores'].map(type_core)
core_median = smartphone['cores'].median()
smartphone['cores'] = smartphone['cores'].fillna(core_median).astype(int)
smartphone['clock_speed'] = smartphone['processor'].str.extract(r'(\d\.?\d*)\s*GHz')[0].astype(float)
speed_median = smartphone['clock_speed'].median()
smartphone['clock_speed'] = smartphone['clock_speed'].fillna(speed_median)

In [131]:
smartphone.dropna(subset=['price_usd'], inplace=True)
smartphone = smartphone[smartphone['processor_name'] != 'Unknown']

In [132]:
camera_swapped = smartphone['card'].str.contains(r'MP|Rear|Front', na=False, case=False)
smartphone.loc[camera_swapped, 'camera'] = smartphone.loc[camera_swapped, 'card']
smartphone.loc[camera_swapped, 'card'] = 'Not Supported'
smartphone['card'] = smartphone['card'].str.replace(r'Memory Card Not Supported', 'Not Supported')

In [133]:
os_pattern = r'(Android|iOS|HarmonyOS|EMUI|Pragati|Hongmeng)'
os_card = smartphone['card'].str.contains(os_pattern, na=False, case=False)
smartphone.loc[os_card, 'os'] = smartphone.loc[os_card, 'card']
smartphone.loc[os_card, 'card'] = 'Not Supported'
smartphone['card'] = smartphone['card'].str.replace(r'Memory Card Not Supported', 'Not Supported', regex=True)

/tmp/ipython-input-2815068116.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  os_card = smartphone['card'].str.contains(os_pattern, na=False, case=False)


In [134]:
smartphone['battery_capacity'] = smartphone['battery'].str.extract(r'(\d+)\s*mAh').astype(float)
smartphone['power_charging'] = smartphone['battery'].str.extract(r'(\d+)\s*W').astype(float).fillna(0)
smartphone['power_charging'] = smartphone['power_charging'].replace(0.0,smartphone['power_charging'].median())

In [135]:
temp_cam = smartphone['camera'].str.split('&', expand=True)
if temp_cam.shape[1] == 1:
    temp_cam[1] = None
temp_cam.columns = ['rear','front']
smartphone['primary_camera_mp'] = temp_cam['rear'].str.extract(r'(\d+)\s*MP')[0].astype(float)
smartphone['total_lens'] = temp_cam['rear'].str.extract(r'(Single|Dual|Triple|Quad)')[0].fillna('Single')
lens_type = {
    'Single':1,
    'Dual':2,
    'Triple':3,
    'Quad':4
}
smartphone['total_lens'] = smartphone['total_lens'].map(lens_type).fillna(1).astype(int)
smartphone['front_camera_mp'] = temp_cam['front'].str.extract(r'(\d+)')[0].astype(float).fillna(0)

In [136]:
smartphone['os_type'] = smartphone['os'].str.extract(r'([a-zA-Z]+)')
smartphone['os_version'] = smartphone['os'].str.extract(r'[vV](\d+)')
smartphone['os_version'] = pd.to_numeric(smartphone['os_version'], errors='coerce').fillna(0).astype(int)

In [137]:
smartphone['screen_size'] = smartphone['display'].str.extract(r'(\d+\.?\d*)\s*inches').astype(float)
smartphone['screen_size'] = smartphone['screen_size'].fillna(smartphone['screen_size'].median())

smartphone[['resolution_width','resolution_height']] = smartphone['display'].str.extract(r'(\d+)\s*[xX]\s*(\d+)')
smartphone['resolution_width'] = pd.to_numeric(smartphone['resolution_width'], errors='coerce')
smartphone['resolution_height'] = pd.to_numeric(smartphone['resolution_height'], errors='coerce')

smartphone['refresh_rate'] = pd.to_numeric(smartphone['display'].str.extract(r'(\d+)\s*Hz')[0],errors='coerce')
smartphone['refresh_rate'] = smartphone['refresh_rate'].fillna(60).astype(int)

smartphone['total_pixels'] = smartphone['resolution_width'] * smartphone['resolution_height']
smartphone['total_pixels'] = smartphone['total_pixels'].fillna(smartphone['total_pixels'].median())

In [ ]:
smartphone['os_type'] = smartphone['os'].str.extract(r'([a-zA-Z]+)')

In [139]:
smartphone.drop(columns=['model','price','sim','processor','ram','display','card','camera','os','battery'],inplace=True)

In [140]:
smartphone = smartphone[~smartphone['os_type'].isin(['Memory', 'RTOS', 'Bluetooth', 'No'])]

In [160]:
smartphone['os_type'] = smartphone['os_type'].replace('Hongmeng', 'Harmony')
smartphone['processor_name'] = smartphone['processor_name'].replace('Apple', 'Bionic')

In [165]:
smartphone['brand'].value_counts()

,count
brand,
Xiaomi,134
Samsung,118
Vivo,111
Realme,95
OPPO,60
Motorola,52
Apple,43
OnePlus,42
iQOO,32


In [157]:
smartphone.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, 0 to 1018
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             937 non-null    float64
 1   brand              937 non-null    object 
 2   price_usd          937 non-null    float64
 3   ram_gb             937 non-null    float64
 4   storage_gb         937 non-null    float64
 5   card_gb            937 non-null    float64
 6   has_5g             937 non-null    int64  
 7   has_nfc            937 non-null    int64  
 8   total_sim          937 non-null    int64  
 9   processor_name     937 non-null    object 
 10  cores              937 non-null    int64  
 11  clock_speed        937 non-null    float64
 12  battery_capacity   928 non-null    float64
 13  power_charging     937 non-null    float64
 14  primary_camera_mp  928 non-null    float64
 15  total_lens         937 non-null    int64  
 16  front_camera_mp    937 non-nul

In [147]:
prestige_map = {
    'apple': 10, 'vertu': 10,
    'samsung': 8, 'google': 8, 'sony': 8,
    'oneplus': 7, 'oppo': 6, 'vivo': 6,
    'xiaomi': 4, 'poco': 3, 'realme': 3, 'infinix': 2
}

smartphone['brand_prestige'] = smartphone['brand'].str.lower().map(prestige_map).fillna(5)

In [149]:
smartphone['rating'] = smartphone['rating'].fillna(smartphone['rating'].median())

In [166]:
smartphone = pd.get_dummies(smartphone, columns=['brand','os_type','processor_name'], prefix=['hp','os','chip'],dtype=int)

In [167]:
smartphone.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, 0 to 1018
Data columns (total 81 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             937 non-null    float64
 1   price_usd          937 non-null    float64
 2   ram_gb             937 non-null    float64
 3   storage_gb         937 non-null    float64
 4   card_gb            937 non-null    float64
 5   has_5g             937 non-null    int64  
 6   has_nfc            937 non-null    int64  
 7   total_sim          937 non-null    int64  
 8   cores              937 non-null    int64  
 9   clock_speed        937 non-null    float64
 10  battery_capacity   928 non-null    float64
 11  power_charging     937 non-null    float64
 12  primary_camera_mp  928 non-null    float64
 13  total_lens         937 non-null    int64  
 14  front_camera_mp    937 non-null    float64
 15  os_version         937 non-null    int64  
 16  screen_size        937 non-nul

In [168]:
smartphone.to_csv('smartphone_cleaned.csv',index=False)